# Bsky integration

In [ ]:
//| export

import { BskyXRPC } from "@mary/bluesky-client";
import type {
  AppBskyFeedDefs,
  AppBskyFeedPost,
} from "@mary/bluesky-client/lexicons";

type Thread = AppBskyFeedDefs.ThreadViewPost;
type Post = AppBskyFeedDefs.PostView;
type PostRecord = AppBskyFeedPost.Record;

const rpc = new BskyXRPC({ service: "https://public.api.bsky.app" });

# Post URL to @at protocol URI

Figure out how to convert bsky.app URL to a URI we can use to fetch data. We
need to go from this:

```
https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d
```

To smth like this

```
at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"
```

`did:plc:ubdeopbbkbgedccgbum7dhsh` is DID for callmephilip.com handle, which
needs to be resolved first

In [ ]:
//| export

const postURLToAtpURI = async (
  postUrl: string,
): Promise<string> => {
  const urlParts = new URL(postUrl);
  const pathParts = urlParts.pathname.split("/");
  const h = await rpc.get("com.atproto.identity.resolveHandle", {
    params: {
      handle: pathParts[2],
    },
  });

  return `at://${h.data.did}/app.bsky.feed.post/${pathParts[4]}`;
};

In [ ]:
await postURLToAtpURI(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

"at://did:plc:ubdeopbbkbgedccgbum7dhsh/app.bsky.feed.post/3lcskn64bss2d"

# Grabbing thread data

Thread has a bunch of nested posts inside replies. Unwrap this into a list of
posts

In [ ]:
//| export

const unwrapThreadPosts = (
  thread: Thread,
): Post[] => {
  const posts: Post[] = [];

  // Add root post
  if (thread.post) {
    posts.push(thread.post);
  }

  // Recursively handle replies
  if (thread.replies) {
    thread.replies.forEach((reply) => {
      posts.push(...unwrapThreadPosts(reply as Thread));
    });
  }

  // Handle nested reply if present
  if ("parent" in thread && thread.parent) {
    posts.push(
      ...unwrapThreadPosts(thread.parent as Thread),
    );
  }

  return posts;
};

export const downloadThread = async (
  postUrl: string,
) => {
  const d = await rpc.get("app.bsky.feed.getPostThread", {
    params: {
      uri: await postURLToAtpURI(postUrl),
    },
  });
  const r = unwrapThreadPosts(d.data.thread as Thread);
  return r.sort(
    (a: Post, b: Post) =>
      new Date((a.record as PostRecord).createdAt).getTime() -
      new Date((b.record as PostRecord).createdAt).getTime(),
  );
};

In [ ]:
const posts = await downloadThread(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

# Converting posts to MD

The most basic setup is when there is just a piece of text display. More complex
cases will have links, attachments

In [ ]:
//| export

export const postToMd = (post: Post): string => {
  // console.log(">>>>>>> working on", post);

  const record = post.record as PostRecord;
  const text = record.text;
  let richtext = text;

  if (record.facets) {
    for (const facet of record.facets) {
      for (const feature of facet.features) {
        if (feature.$type === "app.bsky.richtext.facet#link") {
          const linkPlaceholder = text.slice(
            facet.index.byteStart,
            facet.index.byteEnd,
          );
          richtext = richtext.replace(
            linkPlaceholder,
            `[${linkPlaceholder}](${feature.uri})`,
          );
        }
      }
    }
  }

  return `
    # ${post.author.displayName} (@${post.author.handle}) - ${record.createdAt}

    ${richtext}
  `;
};

// await Deno.jupyter.display(
//   {
//     "text/markdown": postToMd(posts[1]),
//   },
//   { raw: true }
// );

In [ ]:
"Jupyter kernel for Deno - Deno docs docs.deno.com/runtime/refe...".slice(
  36,
  65,
);

"docs.deno.com/runtime/refe..."

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[0]),
  },
  { raw: true },
);


    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z

    🧵 Notes on Deno + Jupyter 🦕 🔭
  

Support links inside posts

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[0]),
  },
  { raw: true },
);


    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z

    🧵 Notes on Deno + Jupyter 🦕 🔭
  

In [ ]:
await Deno.jupyter.display(
  {
    "text/markdown": postToMd(posts[1]),
  },
  { raw: true },
);


    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:49:32.248Z

    Jupyter kernel for Deno - Deno docs [docs.deno.com/runtime/refe...](https://docs.deno.com/runtime/reference/cli/jupyter/)
  

In [ ]:
//| export

export const downloadPostToMd = async (postUrl: string): Promise<string> => {
  const posts = await downloadThread(postUrl);

  return posts.reduce((acc, post) => {
    return acc + postToMd(post);
  }, "");
};

In [ ]:
await downloadPostToMd(
  "https://bsky.app/profile/callmephilip.com/post/3lcskn64bss2d",
);

"\n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:46:20.768Z\n" +
  "\n" +
  "    🧵 Notes on Deno + Jupyter 🦕 🔭\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:49:32.248Z\n" +
  "\n" +
  "    Jupyter kernel for Deno - Deno docs [docs.deno.com/runtime/refe...](https://docs.deno.com/runtime/reference/cli/jupyter/)\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T15:56:32.679Z\n" +
  "\n" +
  "    Repro of original Deno2 jupyter demo with broken mermaid integration - [gist.github.com/aaronblondea...](https://gist.github.com/aaronblondeau/02f94256aab61f409d1f820cc80ea571)\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:00:14.858Z\n" +
  "\n" +
  "    Rich displays for jupyter js kernels - [deno.land/x/display@v1...](https://deno.land/x/display@v1.1.2)\n" +
  "  \n" +
  "    # Philip Nuzhnyi (@callmephilip.com) - 2024-12-08T16:26:17.631Z\n" +
  "\n" +
  "    @kylekelley.bsky.social has a bunch o